# Object detection

Object detection + segmentation using ultralytics [YOLO](https://github.com/ultralytics/ultralytics)

In [1]:
import os
import pickle

from tqdm import tqdm
from ultralytics import YOLO

## Performing inference and storing predictions

Predicting using weights of YOLO 11x for segmentation trained on [COCO's](https://cocodataset.org/#home) dataset.

In [6]:
type = 'nat'

model = YOLO('models/segmentation/yolo11x-seg.pt')
model.to('cuda')

image_files = [os.path.join(f'images/images_{type}', f) for f in os.listdir(f'images/images_{type}') if os.path.isfile(os.path.join(f'images/images_{type}', f))]

results = {}

empty_predictions = 0
prediction_num = []
detected_objects = []
for image_file in tqdm(image_files, total=len(image_files)):
    results[image_file[image_file.rfind('\\')+1:]] = model.predict(image_file, verbose=False)[0]

    if len(results[image_file[image_file.rfind('\\')+1:]].boxes.cls) == 0:
        empty_predictions += 1
    else:
        prediction_num.append(len(results[image_file[image_file.rfind('\\')+1:]].boxes.cls))

    dect_objects = list(results[image_file[image_file.rfind('\\')+1:]].boxes.cls.cpu().numpy())
    for obj in dect_objects:
        if obj not in detected_objects:
            detected_objects.append(obj)

with open('predictions/saved_predictions.pkl', 'wb') as f:
    pickle.dump(results, f)

100%|██████████| 200/200 [00:06<00:00, 30.01it/s]


## Visualizing Results

In [5]:
print(f'Empty predictions: {empty_predictions} ({empty_predictions / len(image_files) * 100:.2f}%)')
print(f'Average number of predictions: {sum(prediction_num) / len(prediction_num):.2f}')
print(f'Unique detected objects: {len(detected_objects)}')

with open('predictions/saved_predictions.pkl', 'rb') as f:
    results = pickle.load(f)

results[list(results.keys())[4]].show()

Empty predictions: 81 (40.50%)
Average number of predictions: 3.44
Unique detected objects: 45
